# Hardware lab 1 (and 2): implementing your control law / Steer by wire

In this first hardware lab we are first going to work without the passive arms of our robot and only focus on the motors. The objective of the lab is to informally play with the motor API and understand the possible issues that can  arise when trying to control accurately a motor.

Concretely, we will play with the notion of PD control (discussed in next week's lecture) and use this to reproduce a [steer-by-wire](https://en.wikipedia.org/wiki/Steer-by-wire) system. 



## Preliminaries
First, listen to Garry or Katy for the first few minutes of the lab while they provide some guidance on the platform.

## Communicating with the device
The DICE machines are connected to the platform using a CAN device. This is hidden within a high-level API but you can have a look at the code if you are interested in seeing how all of this works.

To work, open a python console, or edit a file and execute the code that follows. The CAN bus protocol does not work very with Jupyter notebooks unfortunately and keyboard interruptions won't work.



The API is presented in [AROMotorControlAPI.md](https://github.com/ediadvancedrobotics/hardwarelab/blob/main/README.md). The only methods that we need are `readPosition` and `applyTorqueToMotor`.

Each method has a `motorid` parameter, taking the value 1 or 2, which points to the motor that you wish to control.


Therefore, to read the position (joint angle value) or a motor, simply call:



In [2]:
from motor_control.AROMotorControl import AROMotorControl

mc = AROMotorControl()
mc.readPosition(motorid=1)

343.29999999998836

The returned value is an angle value in degrees, between 0 and 360.


## A torque controlled motor
The motor is controlled by sending a current command to it, which results in its rotation. The rotational force produced is called a torque (more on this during the lectures).


### Torque vs current
This paragraph is just for general knowledge. The torque generated is proportional to the current through the relationship $\tau = k I $, with $\tau$ the torque, $I$ the current (in Amperes), and k a constant given by the manufacturer. In our case, the torque constant provided by the [manufacturer](https://www.myactuator.com/product-page/rmd-l-5010) is $0.16$, which trivially gives you $I = \frac{\tau}{0.16}$. We will trust the manufacturer although this might change from a motor to the other (and is yet another source of approximation).


### Sending a command
To send current command you are required to use an exception handling mechanismthat always safely terminates by resetting the control command. Unfortunately this can't be encapsulated in a function as this results in packages loss.

The `run_until` method defined there also allows you to call N times the same method while ensuring an accurate delay `dt` expires before being called again.

Let us use `run_until` to send a constant torque of 0.02 Nm for 1 seconds



In [8]:
from template import run_until

dt = 0.005
N = int(1. / dt)

print(N)

try:
    run_until(mc.applyTorqueToMotor, N=N, dt=0.005, motorid=1, torque=0.02)
except KeyboardInterrupt:
    print("KeyboardInterrupt received, stopping motors...")
except Exception as e:
    print(f"an error occurred: {e}")
finally:
    mc.applyCurrentToMotor(1, 0)
    mc.applyCurrentToMotor(2, 0)
    print("motors stopped!")

200
times [0.005132215999765322, 0.005254667001281632, 0.005101456001284532, 0.005230685997958062, 0.005331718999514123, 0.0051231109973741695, 0.005064000997663243, 0.005082030998892151, 0.005132778998813592, 0.005240506998234196, 0.005100999998830957, 0.005232744999375427, 0.00502993299960508, 0.0052977049999753945, 0.005208686001424212, 0.00538992699875962, 0.00514837100126897, 0.005053072000009706, 0.005056658999819774, 0.005355255998438224, 0.005200589002924971, 0.0053603129999828525, 0.005056680998677621, 0.0051627639986691065, 0.0053548629985016305, 0.005360229999496369, 0.005393584000557894, 0.005294200000207638, 0.005264771003567148, 0.0053521189984166995, 0.00525346799986437, 0.005302113000652753, 0.005396950997237582, 0.005080340000858996, 0.005231564999121474, 0.005323353998392122, 0.005236888002400519, 0.005043616001785267, 0.005251013000815874, 0.005069251001259545, 0.005256338001345284, 0.005087669000204187, 0.005015493999962928, 0.005003648002457339, 0.00536436000038520



## Question 1
Write a function that performs one iteration of a control loop: Given a desired position and a motor, read the current state and perform one step of PD control to bring the motor to the desired state. 
<mark>Important:</mark> Please make sure that you execute this or any other function in a try-catch block (as shown in [template.py](./template.py)) so that if for any reason you have to terminate your program, the motors stop. Otherwise the motors will continue to execute the last command applied to them.


## Question 2

Test this function within a loop and plot the obtained trajectory. Tune your P and D gains accordingly until you reach a desired behaviour. We are going to control the system at a frequency of 1 khz, which means that each tick should last 1 ms. As for the rununtil function in the lab, the control loop should have a structure like this one:

In [3]:
import time
t = time.perf_counter()
N=int(10e3) #10 seconds
dt = 1. / 1e3
wait = 1. / 1e4
for i in range(N):
    t +=dt
    #run your code
    while(time.perf_counter()-t<dt):
        pass
        time.sleep(wait)

You should of course tune your gains accordingly for the rest of the labs, but this is also the occasion to voluntarily play with the values and observe what happens when you obtain oscillatory behaviours for instance. This will become interesting in particular for the next question.

## Question 3
Now, write a 30s control loop that does the following:
+ Motor 1 is configured to track the position of motor 2
+ Motor 2 is configured to track the position of motor 1

Manually mess around with the motors while the loop is running, you should feel a haptic feedback. A similar system is implemented in some of the recent cars where the steering wheel and the wheels are no longer mechanically connected.